# 中证500ETF期权策略分析

本笔记本用于分析我们的期权策略：每月初卖出中证500ETF平值看涨期权，到期前7天展期。我们将分析这个策略在一年内的表现，包括收益率、风险指标和交易统计。

In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import sys
import os

# 设置绘图风格
plt.style.use('ggplot')
sns.set(font_scale=1.2)

# 添加项目根目录到路径
sys.path.append('..')

# 导入自定义模块
from backtest.engine import run_backtest
from data.data_fetcher import DataFetcher

## 1. 数据准备

首先，我们需要获取中证500ETF和相关期权的数据。如果数据已经存在，我们将直接加载；否则，我们将从Tushare获取数据。

In [ ]:
# 检查数据文件是否存在
etf_data_file = '../data/stock_data.csv'
option_data_file = '../data/option_data.csv'

if not (os.path.exists(etf_data_file) and os.path.exists(option_data_file)):
    # 如果数据不存在，则获取数据
    # 注意：需要设置TUSHARE_TOKEN环境变量或在此处提供token
    token = os.environ.get('TUSHARE_TOKEN')
    if token is None:
        token = input('请输入Tushare API token: ')
    
    fetcher = DataFetcher(token=token)
    
    # 获取2022年的数据
    start_date = '20220101'
    end_date = '20221231'
    
    print(f"正在获取{start_date}至{end_date}的数据...")
    etf_data, option_data = fetcher.prepare_backtest_data(start_date, end_date)
    
    # 保存数据
    fetcher.save_data(etf_data, option_data, etf_file=etf_data_file, option_file=option_data_file)
else:
    print("数据文件已存在，将直接加载数据")

## 2. 查看数据

让我们先看看ETF和期权数据的基本情况。

In [ ]:
# 加载ETF数据
etf_df = pd.read_csv(etf_data_file)
etf_df['trade_date'] = pd.to_datetime(etf_df['trade_date'])

# 显示ETF数据基本信息
print("ETF数据基本信息：")
print(f"数据时间范围：{etf_df['trade_date'].min()} 至 {etf_df['trade_date'].max()}")
print(f"交易日数量：{len(etf_df)}")
print("\n前5行数据：")
etf_df.head()

In [ ]:
# 加载期权数据
option_df = pd.read_csv(option_data_file)
option_df['trade_date'] = pd.to_datetime(option_df['trade_date'])
option_df['expire_date'] = pd.to_datetime(option_df['expire_date'])

# 显示期权数据基本信息
print("期权数据基本信息：")
print(f"期权合约数量：{option_df['ts_code'].nunique()}")
print(f"看涨期权数量：{option_df[option_df['call_put'] == 'C']['ts_code'].nunique()}")
print(f"看跌期权数量：{option_df[option_df['call_put'] == 'P']['ts_code'].nunique()}")
print("\n前5行数据：")
option_df.head()

## 3. 可视化ETF价格走势

让我们绘制中证500ETF在回测期间的价格走势图。

In [ ]:
plt.figure(figsize=(14, 7))

# 绘制ETF收盘价走势
plt.plot(etf_df['trade_date'], etf_df['close'], label='中证500ETF收盘价')

# 添加标题和标签
plt.title('中证500ETF价格走势', fontsize=15)
plt.xlabel('日期')
plt.ylabel('价格')
plt.grid(True)
plt.legend()

# 优化x轴日期显示
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

## 4. 运行策略回测

现在，我们将运行我们的期权策略回测，并分析结果。

In [ ]:
# 运行回测
results = run_backtest(
    etf_data_file=etf_data_file,
    option_data_file=option_data_file,
    cash=100000.0,  # 初始资金10万
    commission=0.0003,  # 佣金率0.03%
    days_before_expiry=7,  # 到期前7天展期
    printlog=True  # 打印日志
)

## 5. 分析回测结果

让我们分析回测结果，包括收益率、交易统计等。

In [ ]:
if results:
    # 基本回测结果
    print(f"初始资金: {results['initial_cash']:.2f}元")
    print(f"最终资金: {results['final_cash']:.2f}元")
    print(f"总收益率: {results['returns']:.2f}%")
    print(f"总交易次数: {len(results['trades'])}次")
    
    # 如果有交易记录，计算更多统计数据
    if results['trades']:
        trades_df = pd.DataFrame(results['trades'])
        
        # 计算平均持仓天数
        avg_days_held = trades_df['days_held'].mean()
        print(f"平均持仓天数: {avg_days_held:.2f}天")
        
        # 计算平均每笔交易收益
        avg_profit = trades_df['profit_loss'].mean()
        print(f"平均每笔交易收益: {avg_profit:.2f}元")
        
        # 计算胜率
        win_rate = (trades_df['profit_loss'] > 0).mean() * 100
        print(f"胜率: {win_rate:.2f}%")
        
        # 计算盈亏比
        profit_trades = trades_df[trades_df['profit_loss'] > 0]
        loss_trades = trades_df[trades_df['profit_loss'] < 0]
        
        if not loss_trades.empty and not profit_trades.empty:
            avg_profit = profit_trades['profit_loss'].mean()
            avg_loss = abs(loss_trades['profit_loss'].mean())
            profit_loss_ratio = avg_profit / avg_loss if avg_loss != 0 else float('inf')
            print(f"盈亏比: {profit_loss_ratio:.2f}")
else:
    print("回测失败，未能获取结果")

## 6. 可视化交易结果

让我们绘制交易结果的可视化图表。